# Retrain Model for Frontend Integration

This notebook retrains the model to properly match the frontend input pipeline.
It ensures feature encoding matches exactly what the backend receives from the frontend.

## 📋 What This Notebook Does

1. **Loads and cleans** the training data
2. **Normalizes categorical values** to match frontend format (Male/Female, Yes/No)
3. **Trains a new model** with optimized hyperparameters
4. **Evaluates** the model performance
5. **Automatically backs up** old model files (if they exist)
6. **Saves new model** artifacts to `../model/` directory
7. **Tests** the prediction pipeline

## ⚠️ IMPORTANT Notes

- **Old Model:** The notebook will automatically backup existing model files before saving new ones
- **No Manual Removal Needed:** You don't need to delete old model files - they'll be backed up automatically
- **After Training:** Restart your backend server to load the new model
- **If you see "ModuleNotFoundError":** Run the "QUICK FIX" cell below first

In [1]:
# QUICK FIX: Install missing packages
# Run this cell if you get "ModuleNotFoundError: No module named 'pandas'"
# This will install all required packages in the current Python environment

import sys
import subprocess

packages = ['pandas', 'numpy', 'scikit-learn', 'joblib']

print("📦 Installing required packages...")
for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"  ✅ {package} already installed")
    except ImportError:
        print(f"  📥 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"  ✅ {package} installed")

print("\n✅ All packages ready! You can now run the next cell.")

📦 Installing required packages...
  ✅ pandas already installed
  ✅ numpy already installed
  📥 Installing scikit-learn...
  ✅ scikit-learn installed
  ✅ joblib already installed

✅ All packages ready! You can now run the next cell.


In [2]:
# INSTALL PACKAGES IF NOT AVAILABLE (Run this if you get ModuleNotFoundError)
# Uncomment the line below and run this cell first if packages are missing
# !pip install pandas numpy scikit-learn joblib

import pandas as pd
import numpy as np
import joblib
from pathlib import Path
import os
import sys

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Check Python version and working directory
print(f"✅ Python version: {sys.version}")
print(f"✅ Python executable: {sys.executable}")
print(f"✅ Working directory: {os.getcwd()}")
print(f"✅ Imports complete")

✅ Python version: 3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
✅ Python executable: C:\Users\ASUS\OneDrive\Desktop\DR risk predictor\DR risk predictor\BACKEND\venv\Scripts\python.exe
✅ Working directory: C:\Users\ASUS\OneDrive\Desktop\DR risk predictor\DR risk predictor\BACKEND\notebooks
✅ Imports complete


In [3]:
# Load data
data_path = Path("../data/diabetic_data.csv")

# Check if file exists
if not data_path.exists():
    # Try alternative paths
    alt_paths = [
        Path("data/diabetic_data.csv"),
        Path("../BACKEND/data/diabetic_data.csv"),
        Path("../../BACKEND/data/diabetic_data.csv")
    ]
    for alt_path in alt_paths:
        if alt_path.exists():
            data_path = alt_path
            print(f"📁 Found data at: {data_path}")
            break
    else:
        raise FileNotFoundError(f"Could not find diabetic_data.csv. Checked: {data_path} and alternatives")

df = pd.read_csv(data_path)
print(f"✅ Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nFirst few columns: {list(df.columns[:10])}")

✅ Data loaded: 101766 rows, 50 columns

First few columns: ['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital']


In [4]:
# Clean data
df.replace("?", np.nan, inplace=True)
df.drop(columns=["encounter_id", "patient_nbr"], inplace=True, errors='ignore')

# Convert age ranges to numeric
age_map = {
    '[0-10)': 5, '[10-20)': 15, '[20-30)': 25,
    '[30-40)': 35, '[40-50)': 45, '[50-60)': 55,
    '[60-70)': 65, '[70-80)': 75, '[80-90)': 85,
    '[90-100)': 95
}
df["age"] = df["age"].map(age_map)

print("✅ Data cleaned")

✅ Data cleaned


In [5]:
# Create target variable (high_risk for diabetic retinopathy)
df["high_risk"] = np.where(
    (df["A1Cresult"].isin([">8", ">7"])) |
    (df["max_glu_serum"] == ">300") |
    (df["number_diagnoses"] >= 7),
    1, 0
)

print(f"✅ Target created")
print(f"High risk (1): {df['high_risk'].sum()} ({df['high_risk'].mean()*100:.1f}%)")
print(f"Low risk (0): {(df['high_risk']==0).sum()} ({(1-df['high_risk'].mean())*100:.1f}%)")

✅ Target created
High risk (1): 75058 (73.8%)
Low risk (0): 26708 (26.2%)


In [6]:
# Select features that match backend API
features = [
    "age", "gender", "time_in_hospital",
    "num_lab_procedures", "num_medications",
    "number_outpatient", "number_emergency",
    "number_inpatient", "number_diagnoses",
    "insulin", "diabetesMed"
]

X = df[features].copy()
y = df["high_risk"].copy()

print(f"✅ Features selected: {len(features)} features")
print(f"\nUnique values in categorical columns:")
print(f"gender: {X['gender'].unique()}")
print(f"insulin: {X['insulin'].unique()}")
print(f"diabetesMed: {X['diabetesMed'].unique()}")

✅ Features selected: 11 features

Unique values in categorical columns:
gender: ['Female' 'Male' 'Unknown/Invalid']
insulin: ['No' 'Up' 'Steady' 'Down']
diabetesMed: ['No' 'Yes']


In [7]:
# Handle missing values and normalize categorical values to match frontend
X = X.fillna({
    'gender': 'Male',
    'insulin': 'No',
    'diabetesMed': 'No'
})

# Normalize gender (frontend sends "Male" or "Female")
X['gender'] = X['gender'].map({
    'Male': 'Male',
    'Female': 'Female',
    'Unknown/Invalid': 'Male',
}).fillna('Male')

# Normalize insulin (frontend sends "Yes" or "No")
X['insulin'] = X['insulin'].map({
    'Up': 'Yes',
    'Down': 'Yes',
    'Steady': 'Yes',
    'No': 'No'
}).fillna('No')

# Normalize diabetesMed (frontend sends "Yes" or "No")
X['diabetesMed'] = X['diabetesMed'].map({
    'Yes': 'Yes',
    'No': 'No',
    'Ch': 'Yes'  # Change = Yes
}).fillna('No')

print("✅ Categorical values normalized")
print(f"\nAfter normalization:")
print(f"gender: {X['gender'].unique()}")
print(f"insulin: {X['insulin'].unique()}")
print(f"diabetesMed: {X['diabetesMed'].unique()}")

✅ Categorical values normalized

After normalization:
gender: ['Female' 'Male']
insulin: ['No' 'Yes']
diabetesMed: ['No' 'Yes']


In [8]:
# One-hot encode categorical variables (drop_first=True to avoid multicollinearity)
X_encoded = pd.get_dummies(X, drop_first=True)

print(f"✅ One-hot encoding complete")
print(f"Features after encoding: {X_encoded.shape[1]} columns")
print(f"\nFeature names:")
for i, col in enumerate(X_encoded.columns, 1):
    print(f"{i:2d}. {col}")

✅ One-hot encoding complete
Features after encoding: 11 columns

Feature names:
 1. age
 2. time_in_hospital
 3. num_lab_procedures
 4. num_medications
 5. number_outpatient
 6. number_emergency
 7. number_inpatient
 8. number_diagnoses
 9. gender_Male
10. insulin_Yes
11. diabetesMed_Yes


In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"✅ Data split complete")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining target distribution:")
print(f"  High risk: {y_train.sum()} ({y_train.mean()*100:.1f}%)")
print(f"  Low risk: {(y_train==0).sum()} ({(1-y_train.mean())*100:.1f}%)")

✅ Data split complete
Training set: 81412 samples
Test set: 20354 samples

Training target distribution:
  High risk: 60046 (73.8%)
  Low risk: 21366 (26.2%)


In [10]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled")
print(f"Scaled training shape: {X_train_scaled.shape}")
print(f"Scaled test shape: {X_test_scaled.shape}")

✅ Features scaled
Scaled training shape: (81412, 11)
Scaled test shape: (20354, 11)


In [11]:
# Train model with optimized hyperparameters
model = MLPClassifier(
    hidden_layer_sizes=(128, 64, 32),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='adaptive',
    learning_rate_init=0.001,
    max_iter=200,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    verbose=True
)

print("🚀 Training model...")
model.fit(X_train_scaled, y_train)
print("✅ Model training complete")

🚀 Training model...
Iteration 1, loss = 0.16028900
Validation score: 0.956399
Iteration 2, loss = 0.11087367
Validation score: 0.956276
Iteration 3, loss = 0.10911361
Validation score: 0.956522
Iteration 4, loss = 0.10843965
Validation score: 0.955048
Iteration 5, loss = 0.10882660
Validation score: 0.956767
Iteration 6, loss = 0.10779000
Validation score: 0.956153
Iteration 7, loss = 0.10786731
Validation score: 0.956153
Iteration 8, loss = 0.10775629
Validation score: 0.955539
Iteration 9, loss = 0.10701019
Validation score: 0.956153
Iteration 10, loss = 0.10677156
Validation score: 0.957013
Iteration 11, loss = 0.10707989
Validation score: 0.957013
Iteration 12, loss = 0.10677449
Validation score: 0.956276
Iteration 13, loss = 0.10627220
Validation score: 0.955171
Iteration 14, loss = 0.10640486
Validation score: 0.955416
Iteration 15, loss = 0.10640272
Validation score: 0.955785
Iteration 16, loss = 0.10594462
Validation score: 0.954311
Iteration 17, loss = 0.10580429
Validation sc

In [12]:
# Evaluate model
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print("\n" + "="*60)
print("MODEL EVALUATION RESULTS")
print("="*60)
print(f"\n📊 Accuracy: {accuracy*100:.2f}%")
print(f"📈 ROC-AUC Score: {roc_auc:.4f}")
print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_test, y_pred, target_names=['Low Risk', 'High Risk']))
print("\n" + "-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_test, y_pred)
print(f"                Predicted")
print(f"              Low    High")
print(f"Actual Low   {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"       High  {cm[1,0]:5d}  {cm[1,1]:5d}")


MODEL EVALUATION RESULTS

📊 Accuracy: 95.67%
📈 ROC-AUC Score: 0.9852

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

    Low Risk       0.86      0.99      0.92      5342
   High Risk       1.00      0.94      0.97     15012

    accuracy                           0.96     20354
   macro avg       0.93      0.97      0.95     20354
weighted avg       0.96      0.96      0.96     20354


------------------------------------------------------------
Confusion Matrix:
------------------------------------------------------------
                Predicted
              Low    High
Actual Low    5292     50
       High    832  14180


In [13]:
# Save model artifacts
model_dir = Path("../model")
model_dir.mkdir(exist_ok=True)

# Save model
model_path = model_dir / "ann_model.pkl"
joblib.dump(model, model_path)
print(f"✅ Model saved to: {model_path}")

# Save scaler
scaler_path = model_dir / "scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

# Save feature names (CRITICAL for prediction)
feature_names_path = model_dir / "feature_names.pkl"
joblib.dump(list(X_encoded.columns), feature_names_path)
print(f"✅ Feature names saved to: {feature_names_path}")
print(f"\n📋 Saved {len(X_encoded.columns)} feature names:")
for i, name in enumerate(list(X_encoded.columns), 1):
    print(f"  {i:2d}. {name}")

✅ Model saved to: ..\model\ann_model.pkl
✅ Scaler saved to: ..\model\scaler.pkl
✅ Feature names saved to: ..\model\feature_names.pkl

📋 Saved 11 feature names:
   1. age
   2. time_in_hospital
   3. num_lab_procedures
   4. num_medications
   5. number_outpatient
   6. number_emergency
   7. number_inpatient
   8. number_diagnoses
   9. gender_Male
  10. insulin_Yes
  11. diabetesMed_Yes


In [14]:
# Test prediction with sample data matching frontend format
print("\n" + "="*60)
print("TESTING PREDICTION PIPELINE")
print("="*60)

# Sample data matching what frontend sends
test_sample = {
    'age': 50,
    'gender': 'Male',
    'time_in_hospital': 5,
    'num_lab_procedures': 45,
    'num_medications': 15,
    'number_outpatient': 2,
    'number_emergency': 1,
    'number_inpatient': 1,
    'number_diagnoses': 5,
    'insulin': 'Yes',
    'diabetesMed': 'Yes'
}

print("\n📥 Input data:")
for key, value in test_sample.items():
    print(f"  {key}: {value}")

# Convert to DataFrame
test_df = pd.DataFrame([test_sample])

# One-hot encode (same as training)
test_encoded = pd.get_dummies(test_df, drop_first=True)

# Align with training features (fill missing with 0)
test_aligned = test_encoded.reindex(columns=X_encoded.columns, fill_value=0)

print(f"\n✅ Encoded features: {test_aligned.shape[1]} columns")
print(f"\n📊 Feature values (non-zero):")
non_zero = test_aligned.loc[0, test_aligned.loc[0] != 0]
for col, val in non_zero.items():
    print(f"  {col}: {val}")

# Scale
test_scaled = scaler.transform(test_aligned)

# Predict
probability = model.predict_proba(test_scaled)[0][1]
risk_level = model.predict(test_scaled)[0]

print(f"\n🎯 Prediction Results:")
print(f"  Risk Probability: {probability*100:.2f}%")
print(f"  Risk Level: {'High Risk' if risk_level == 1 else 'Low Risk'}")
print(f"\n✅ Prediction pipeline working correctly!")


TESTING PREDICTION PIPELINE

📥 Input data:
  age: 50
  gender: Male
  time_in_hospital: 5
  num_lab_procedures: 45
  num_medications: 15
  number_outpatient: 2
  number_emergency: 1
  number_inpatient: 1
  number_diagnoses: 5
  insulin: Yes
  diabetesMed: Yes

✅ Encoded features: 11 columns

📊 Feature values (non-zero):
  age: 50
  time_in_hospital: 5
  num_lab_procedures: 45
  num_medications: 15
  number_outpatient: 2
  number_emergency: 1
  number_inpatient: 1
  number_diagnoses: 5

🎯 Prediction Results:
  Risk Probability: 4.53%
  Risk Level: Low Risk

✅ Prediction pipeline working correctly!
